In [ ]:
from glob import glob
# import sys
from dotenv import load_dotenv

load_dotenv()
# sys.path.append("/u/zliu/datastor1/KE-by-CP")
import pandas as pd
# from experiments.musique.inference_only import macro_averaging
from knowledge_propagation.utils import io, vars, extractor
import os
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import describe
from thefuzz import fuzz

from datasets import load_dataset, load_from_disk

from copy import deepcopy

from dateutil.parser import parse
from dateutil.parser import ParserError

from collections import defaultdict

import re


In [62]:
popular_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/benchmark/popular.json")
assert len(popular_examples) == 1
popular_examples = popular_examples[0]

# random_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/benchmark/random.json")
# assert len(random_examples) == 1
# random_examples = random_examples[0]

recent_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/benchmark/recent.json")
assert len(recent_examples) == 1
recent_examples = recent_examples[0]

ripple_edits_examples = popular_examples +  recent_examples # + random_examples
len(ripple_edits_examples)

2833

In [63]:
# random_examples[0]

In [69]:
non_zero_outerloop_count = 0
strong_meta_examples = []
weak_meta_examples = []
for example in ripple_edits_examples:
    outerloop_instances = example["Logical_Generalization"] + example["Compositionality_I"] + example["Compositionality_II"] + example["Subject_Aliasing"]
    # for ins in outerloop_instances:
        # assert len(ins["test_queries"]) == 1
    locality_instances = example["Relation_Specificity"] + example["Forgetfulness"] 
    # non_zero_outerloop_count += len(outerloop_instances) > 0
    if len(outerloop_instances) > 0 and len(locality_instances) > 0:
        
        outerloop_queries = [q for instance in outerloop_instances for q in instance["test_queries"]]
        outerloop_queries = [q for q in outerloop_queries if len(q["answers"]) > 0]
        outerloop_queries = [q for q in outerloop_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in outerloop_queries])
        
        if len(outerloop_queries) == 0:
            continue
        
        locality_queries = [q for instance in locality_instances for q in instance["test_queries"]]
        locality_queries = [q for q in locality_queries if len(q["answers"]) > 0]
        locality_queries = [q for q in locality_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in locality_queries])
        
        assert len(locality_queries) > 0
        
        strong_meta_examples.append(example)
    elif len(locality_instances) == 0:
        
        outerloop_queries = [q for instance in outerloop_instances for q in instance["test_queries"]]
        outerloop_queries = [q for q in outerloop_queries if len(q["answers"]) > 0]
        outerloop_queries = [q for q in outerloop_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in outerloop_queries])
        
        if len(outerloop_queries) == 0:
            continue
        
        weak_meta_examples.append(example)
    else:
        locality_queries = [q for instance in locality_instances for q in instance["test_queries"]]
        locality_queries = [q for q in locality_queries if len(q["answers"]) > 0]
        locality_queries = [q for q in locality_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in locality_queries])
        
        assert len(locality_queries) > 0
        
        weak_meta_examples.append(example)
        
print("# strong examples", len(strong_meta_examples))
print("# weak examples", len(weak_meta_examples))

# strong examples 2080
# weak examples 690


In [70]:
len(strong_meta_examples) + len(weak_meta_examples)

2770

In [76]:
[[a["value"] for a in q["answers"] if len(a["value"].strip()) > 0] for q in locality_queries]

[['Mary Prudence Ord'],
 ['Lawrence Hay Fyffe'],
 ['Henrietta Frances Arnaud Blagden'],
 ['Alan Fyffe', 'Mary Constance Fyffe'],
 ['male'],
 ['history', 'journalism']]

In [71]:
n_test = 200
n_valid = 200
np.random.shuffle(strong_meta_examples)
np.random.shuffle(weak_meta_examples)

test_examples = strong_meta_examples[:n_test]
valid_examples = strong_meta_examples[n_test:n_test+n_valid]
train_examples = strong_meta_examples[n_test+n_valid:] + weak_meta_examples
np.random.shuffle(train_examples)

In [ ]:
# io.dump_jsonlines(train_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train/train.jsonl")
# io.dump_jsonlines(valid_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train/valid.jsonl")
# io.dump_jsonlines(test_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train/test.jsonl")


In [ ]:
train_examples

In [3]:
train_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train/train.jsonl")

In [10]:
non_zero_outerloop_count = 0
strong_meta_examples = []
weak_meta_examples = []
count_rel = defaultdict(int)
count_prop_types = defaultdict(int)

for example in train_examples:
    outerloop_instances = example["Logical_Generalization"] + example["Compositionality_I"] + example["Compositionality_II"] + example["Subject_Aliasing"]
    # for ins in outerloop_instances:
        # assert len(ins["test_queries"]) == 1
    locality_instances = example["Relation_Specificity"] + example["Forgetfulness"] 
    
    outerloop_queries = []
    for k in ["Logical_Generalization", "Compositionality_I", "Compositionality_II", "Subject_Aliasing"]:
        for instance in example[k]:
            for q in instance["test_queries"]:
                if len(q["answers"]) > 0 and len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0:
                    q["question_type"] = k
                    outerloop_queries.append(q)
    for outer_q in outerloop_queries:
        count_prop_types["efficacy::"+ outer_q["question_type"]] += 1
        count_rel["efficacy::"+ outer_q["relation"]] += 1
    # assert len(outerloop_queries) > 0
    
    locality_queries = []
    for k in ["Relation_Specificity", "Forgetfulness"]:
        for instance in example[k]:
            for q in instance["test_queries"]:
                if len(q["answers"]) > 0 and len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0:
                    q["question_type"] = k
                    locality_queries.append(q)
    for loc_q in locality_queries:
        count_prop_types["specificity::"+ loc_q["question_type"]] += 1
        count_rel["specificity::"+ loc_q["relation"]] += 1
    # assert len(locality_queries) > 0
        
# print("# strong examples", len(strong_meta_examples))
# print("# weak examples", len(weak_meta_examples))

In [11]:
count_prop_types

defaultdict(int,
            {'efficacy::Logical_Generalization': 1289,
             'efficacy::Compositionality_I': 9613,
             'specificity::Relation_Specificity': 10679,
             'specificity::Forgetfulness': 1450,
             'efficacy::Subject_Aliasing': 3062,
             'efficacy::Compositionality_II': 1325})

In [ ]:
count_rel

defaultdict(int,
            {'efficacy::SIBLING': 384,
             'efficacy::SPOUSE': 904,
             'specificity::SIBLING': 289,
             'specificity::SEX_OR_GENDER': 1378,
             'specificity::COUNTRY_OF_CITIZENSHIP': 1278,
             'specificity::RELIGION': 326,
             'specificity::SPOUSE': 428,
             'specificity::DIRECTOR': 196,
             'specificity::SCREENWRITER': 158,
             'specificity::COMPOSER': 171,
             'specificity::CAST_MEMBER': 196,
             'efficacy::FOLLOWED_BY': 146,
             'specificity::FIELD_OF_WORK': 211,
             'specificity::FOUNDER': 65,
             'specificity::FOLLOWS': 71,
             'specificity::PLACE_OF_BIRTH': 1079,
             'specificity::PLACE_OF_DEATH': 456,
             'specificity::AWARD_RECEIVED': 546,
             'efficacy::UNCLE': 60,
             'efficacy::AUNT': 47,
             'efficacy::CHILD': 279,
             'efficacy::NUMBER_OF_CHILDREN': 166,
             'e